In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/treesXYZ/meta/META.csv')

In [30]:
sp = 'DEAD'
print(df[(df['sp']==sp) & (df['multistem']==1)].shape[0])
print(df[(df['sp']==sp) & (df['dead']==1)].shape[0])
print(df[(df['sp']==sp) & (df['out_of_bounds']==0)].shape[0])

0
1
0


In [25]:
df

,old_id,id,sp,out_of_bounds,multistem,dead,qa,Unnamed: 7
0,alt01_001,alt01_1,QUEFAG,1,1,0,1,NaN
1,alt01_002,alt01_2,QUEFAG,1,0,0,1,NaN
2,alt01_004,alt01_4,QUEFAG,1,1,0,1,NaN
3,alt01_005,alt01_5,QUEFAG,1,0,0,1,NaN
4,alt01_006,alt01_6,PINSYL,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...
2480,cue06_017,cue06_17,PINPIN,1,0,0,1,NaN
2481,cue06_018,cue06_18,PINPIN,1,0,0,1,NaN
2482,cue06_019,cue06_19,PINPIN,1,0,0,1,NaN
2483,cue06_020,cue06_20,PINSYL,1,1,0,1,NaN
